In [1]:
#using random_state=17 n_estimators=75

import kagglegym
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from time import time

env = kagglegym.make()
o = env.reset()
excl = [env.ID_COL_NAME, env.SAMPLE_COL_NAME, env.TARGET_COL_NAME, env.TIME_COL_NAME]
col = [c for c in o.train.columns if c not in excl]

#train = pd.read_hdf('../input/train.h5')
train = o.train[col]#train[col]
d_mean= train.median(axis=0)

train = o.train[col]
n = train.isnull().sum(axis=1)
for c in train.columns:
    train[c + '_nan_'] = pd.isnull(train[c]) 
    d_mean[c + '_nan_'] = 0
train = train.fillna(d_mean)
train['znull'] = n
n = []

t0=time()
rfr = ExtraTreesRegressor(n_estimators=50, max_depth=4, n_jobs=-1, random_state=17, verbose=0)
model1 = rfr.fit(train, o.train['y'])
print("rfr train", time()-t0)

#https://www.kaggle.com/bguberfain/two-sigma-financial-modeling/univariate-model-with-clip/run/482189
t0=time()
low_y_cut = -0.085
high_y_cut = 0.075
y_is_above_cut = (o.train.y > high_y_cut)
y_is_below_cut = (o.train.y < low_y_cut)
y_is_within_cut = (~y_is_above_cut & ~y_is_below_cut)
model2 = LinearRegression(n_jobs=-1)
model2.fit(np.array(o.train[col].fillna(d_mean).loc[y_is_within_cut, 'technical_20'].values).reshape(-1,1), o.train.loc[y_is_within_cut, 'y'])
train = []
print("lr train", time()-t0)

#https://www.kaggle.com/ymcdull/two-sigma-financial-modeling/ridge-lb-0-0100659
ymean_dict = dict(o.train.groupby(["id"])["y"].median())

coeff1=.7
coeff2=.975

t0=time()
while True:
    test = o.features[col]
    n = test.isnull().sum(axis=1)
    for c in test.columns:
        test[c + '_nan_'] = pd.isnull(test[c])
    test = test.fillna(d_mean)
    test['znull'] = n
    pred = o.target
    test2 = np.array\
    (o.features[col].fillna(d_mean)['technical_20'].values).reshape(-1,1)
    pred['y'] = (model1.predict(test).clip(low_y_cut, high_y_cut) * coeff1)+\
    (model2.predict(test2).clip(low_y_cut, high_y_cut) * (1-coeff1))
    pred['y'] = pred.apply(lambda r: coeff2 * r['y'] + \
                    (1-coeff2) * ymean_dict[r['id']] \
                    if r['id'] in ymean_dict else r['y'], axis = 1)
    pred['y'] = [float(format(x, '.6f')) for x in pred['y']]
    o, reward, done, info = env.step(pred)
    if done:
        print("low=",low_y_cut, "high=", high_y_cut, "coeff1=",\
                 coeff1, "coeff2=", coeff2, \
                 "r=",info["public_score"])
        break
    if o.features.timestamp[0] % 100 == 0:
        print(reward)
        
print("fit", time()-t0)

C:\Users\jiguo\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('rfr train', 127.94800019264221)
('lr train', 1.2669999599456787)


C:\Users\jiguo\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


-0.320104153066
-0.181091702079
-0.171788855014
-0.00278671792259
-0.126934814089
-0.164409295943
-0.210570298281
-0.303572823751
-0.0226639680368
('low=', -0.085, 'high=', 0.075, 'coeff1=', 0.7, 'coeff2=', 0.975, 'r=', 0.022256189956986024)
('fit', 298.70300006866455)
